In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
#hide
import json
from pathlib import Path
from nbdev.showdoc import show_doc
from getpass import getuser, getpass
from fiscaliza.redmine import *
from fiscaliza.constants import SERVICOS
from rich.console import Console
console = Console()

# Fiscaliza

> Scripts para baixar informações sobre inspeções no Sistema Fiscaliza da Anatel e atualizar inspeções por meio da API do Redmine.

## Instalação
Como boa prática é sugerido a criação de um ambiente virtual para encapsular o ambiente python no qual você irá instalar a biblioteca

Utilizando o `conda`:
```bash
conda create -n fiscaliza python=3.8
conda activate fiscaliza
python -m pip install fiscaliza
```
Utilizando somente python:

Windows
```bash
python -m venv <pasta onde criar o ambiente virtual>
<pasta onde criou o ambiente virtual>\Scripts\activate.bat
python -m pip install fiscaliza
```
Linux:
```bash
python -m venv <pasta onde criar o ambiente virtual>
source <pasta onde criou o ambiente virtual>/Scripts/activate
python -m pip install fiscaliza
```

Caso não desejar utilizar um ambiente virtual basta rodar:
`pip install fiscaliza`

## Como utilizar

O `Fiscaliza` da Anatel é um sistema web que utiliza a plataforma Redmine. É basicamente um softaware de gerenciamento e atividades.

O Fiscaliza possui 2 módulos para os fiscais utilizarem:
* Módulo de Teste ( Homologação ) : "https://sistemashm.anatel.gov.br/fiscaliza"
* Módulo de Produção: "https://sistemas.anatel.gov.br/fiscaliza/"

Ambos módulos precisam de um usuário e senha com acesso autorizado, em geral somente servidores da Anatel.

O Escopo desse módulo basicamente está encapsulado em 3 funções básicas:
* detalhar_inspecao - **Retorna as informações do estado atual da Issue ( Inspeção )**
* validar_dicionario - **Para dado dicionário de dados, formata-o para a API do Redmine**
* relatar_inspecao - **Atualiza e Issue com o dicionário de dados passado**

A terceira função `relatar_inspecao` possui escopo limitado. Atualmente somente é formatado e relatado Inspeções ( Issue ) to tipo "Uso do Espectro - Monitoração". Essa inspeção é a principal utilizada para as atividades de monitoração da Anatel e foi o que motivou a criação da presente biblioteca. Outras inspeções possuem campos distintos e assim exigem formatação distinta. Versões futuras poderão abarcar o relato de diferentes inspeções. 

### Detalhar Inspeção

In [ ]:
#hide
show_doc(detalhar_issue)

<h4 id="detalhar_issue" class="doc_header"><code>detalhar_issue</code><a href="https://github.com/ronaldokun/fiscaliza/tree/master/fiscaliza/redmine.py#L489" class="source_link" style="float:right">[source]</a></h4>

> <code>detalhar_issue</code>(**`inspecao`**:"Número da Inspeção a ser relatada", **`login`**:"Login Anatel do Usuário"=*`None`*, **`senha`**:"Senha Utilizada nos Sistemas Interativos da Anatel"=*`None`*, **`fiscaliza`**:"Objeto Redmine logado, opcional ao login e senha"=*`None`*, **`teste`**:"Indica se o relato será de teste"=*`True`*)

Recebe número da inspeção `inspecao`, o login e senha ou opcionalmente objeto Redmine logado `fiscaliza`
inspecao: str - Número da Inspeção a ser relatada
login: str - Login Anatel do Usuário
senha: str - Senha Utilizada nos Sistemas Interativos da
fiscaliza: Redmine - Objeto Redmine logado, opcional ao login e senha
teste: bool - Caso verdadeiro o Fiscaliza de Teste ( Homologação ) é utilizado

Returns:
    dict: Retorna um dicionário com a Situação Atual e campos preenchidos da Inspeção

Vamos exemplificar a Inspeção de Teste `57824`

In [ ]:
login = getuser()
senha = 'D0_0r_d0_n0t' #getpass()

In [ ]:
# issue = '72088'
# detalhes = detalhar_issue(issue, login, senha)
# console.print(detalhes)

In [ ]:
issue = '72145'
detalhes = detalhar_issue(issue, login, senha, teste=True)
console.print(detalhes)

{
    'Anexos': {
        'Lista_de_Exceção_PMEC_Etapa_2_Sorocaba': 'http://sistemashm.anatel.gov.br/fiscaliza/
attachments/download/1303/Lista_de_Exce%C3%A7%C3%A3o_PMEC_Etapa_2_Sorocaba'
    },
    'id': '72145',
    'subject': 'INSP_GR01_2021_0715',
    'status': 'Relatando',
    'priority': 'Normal',
    'start_date': '2021-09-13',
    'due_date': '2021-12-31',
    'description': 'Atendimento da Denúncia AC202010}213075425 (6104512), \r\nverificação da 
Potência e Intensidade de Campo Elétrico da Frequência 105.1MHz e seus harmônicos, \r\nalém 
da checagem de Intermodulação e Espúrios nas frequências 450.3MHz e 750MHz.',
    'Data_de_inicio_efetivo': datetime.date(2021, 8, 11),
    'Classe_da_Inspecao': 'Técnica',
    'Tipo_de_Inspecao': 'Uso do Espectro - Monitoração',
    'Ano': '2021',
    'Coordenacao': '',
    'Numero_Sei_do_Processo': '{"numero"=>"53554.000013/2021-11", "link_acesso"=>"https://sei
hm.anatel.gov.br/sei/controlador.php?acao=procedimento_trabalhar&id_procedimento=1964149"}',
    'Fiscal_Responsavel': 'Ronaldo da Silva Alves Batista',
    'Fiscais': [
        'Ronaldo da Silva Alves Batista',
        'Paulo Diogo Costa',
        'Mario Augusto Volpini'
    ],
    'Entidade_da_Inspecao': [],
    'UF_Municipio': ['SP/Itatiba'],
    'Servicos_da_Inspecao': ['230 - COLETIVO - RADIODIFUSÃO SONORA EM FREQÜÊNCIA MODULADA'],
    'Qnt_de_emissoes_na_faixa': '1',
    'Emissoes_nao_autorizadas': '0',
    'Horas_de_Preparacao': '2',
    'Horas_de_Deslocamento': '0',
    'Horas_de_Execucao': '24',
    'Horas_de_Conclusao': '4',
    'SAV': '',
    'PCDP': '',
    'Procedimentos': [],
    'Latitude': '-23.0175',
    'Longitude': '-46.884722',
    'Uso_de_PF': 'Não se aplica PF - uso apenas de formulários',
    'Acao_de_risco_a_vida_criada': 'Não',
    'Frequencia_Inicial': '102.9',
    'Unidade_da_Frequencia_Inicial': 'MHz',
    'Frequencia_Final': '102.9',
    'Unidade_da_Frequencia_Final': 'MHz',
    'Agrupamento': '',
    'AppFiscaliza': '0',
    'Relatorio_de_Monitoramento': '{"numero"=>"0191899", "link_acesso"=>"https://seihm.anatel
.gov.br/sei/controlador.php?acao=procedimento_trabalhar&id_procedimento=1964149&id_documento=
1964508"}',
    'Reservar_Instrumentos': '0',
    'Utilizou_algum_instrumento': '0',
    'id_ACAO': 72088,
    'nome_ACAO': 'ACAO_GR01_2021_0688',
    'descricao_ACAO': '',
    'Users': [
        'Alexandre Elias de Andrade Oliveira',
        'Alexandre Freitas de Lima',
        'Alexandre Inacio',
        'Alexandre Junzo Hamada',
        'Alfredo de Andrade Filho',
        'Ananias Pereira',
        'Antonio Carlos Cardoso de Mello',
        'Aparecido Sebastiao da Silva',
        'Arthur Pisaruk',
        'Carlos Augusto de Carvalho',
        'Carlos Eduardo Guimaraes Silveira',
        'Carlos da Paixao Filho',
        'Celio Yukio Takahashi',
        'Celso Luiz Maximino',
        'Diogo Caldeira',
        'Ediceu Beraldi',
        'Eduardo Narkevicius',
        'Elcio Maehara',
        'Eustaquio Lages Duarte',
        'Fiscal UD',
        'Gauber Albuquerque',
        'Gilson Ponce Ayres Filho',
        'Helio Lopes de Carvalho Filho',
        'Higor da Paz Melo',
        'Hugo Santana Lima',
        'Humberto Barbosa Vinagre',
        'Jamilson Evangelista',
        'Joao Yokoyama',
        'Joaquim Miranda',
        'José Antônio S. Sanches',
        'Julio Cesar de Assis Santos',
        'Kiyotomo Kawamura',
        'Laert Calil Junior',
        'Lannei Vilela Moraes',
        'Luis Lagomes',
        'Luiz Vinicios Mielniczuk Seelig',
        'Marcelo Augusto Scacabarozi',
        'Marcelo Vaz Netto',
        'Marcio Colazingari',
        'Marcio Costa',
        'Marcio Rodrigues Maciel',
        'Marcos Antônio Rodrigues',
        'Marcos Juliano Valim da Silva',
        'Maria Teresa Flosi Garrafa',
        'Mario Augusto Volpini',
        'Murilo Amaro',
        'Osnir Lopes',
        'Paulo Diogo Costa',
        'Pedro Arai',
        'Renato Sadao Kushioyada',
        'Ricardo San

Agora vamos testar o retorno de informações para um Inspeção no Fiscaliza de produção `teste=False`.

### Validação de Dados e Formatação para submeter à API
A função anterior somente retorna as informações já constantes em dada Inspeção ( Issue ) do Fiscaliza. Para alterarmos ou atualizarmos dada inspeção, precisamos passar um dicionário de dados ou um caminho para um arquivo `.json` ou pickle `.pkl` onde conste esse dicionário de dados serializado. O exemplo seguinte mostra um dicionário de dados com as informações básicas constantes de uma monitoração e a formatação que elas são validadas:

In [ ]:
d = {}

d['Classe_da_Inspecao'] = 'Técnica' # str

d['Tipo_de_Inspecao'] = 'Uso do Espectro - Monitoração' #str

d['description'] = '''Atendimento à Ação ACAO_GR01_2021_0792 #74488, instruída pela Denúncia 202108182925413 (7283673), 
verificação da Potência e Intensidade de Campo Elétrico da Frequência 102.9MHz e seus harmônicos, 
além de Radiogoniometria para checagem se o transmissor está instalado no local autorizado: Morro do Mombuca S/N Itatiba/SP..''' #str

d['Fiscal_Responsavel'] = 'Ronaldo da Silva Alves Batista' #str

d['Fiscais'] = ['Ronaldo da Silva Alves Batista'] #string ou lista de strings

# # Windows
d['Html'] = r'D:\html_simples.html' #str

# #ou 

# #Unix d['Html'] = '/d/OneDrive - ANATEL/Monitoramento/53504.0005432021-55/Guarulhos.html' #str
            
d['Gerar_Relatorio'] = 1 # int 0 ou 1

d['Frequencia_Inicial']  = 102.9 #int ou float

d['Unidade_da_Frequencia_Inicial'] = 'MHz' #string

d['Frequencia_Final'] = 102.9 #int ou float

d['Unidade_da_Frequencia_Final'] = 'MHz' #string

d['start_date'] = '2021-09-13' #YYYY-MM-DD #string nesse formato

d['due_date'] = '2021-12-31'  #YYYY-MM-DD #string nesse formato

d['UF_Municipio'] = "SP/Itatiba" # string ou Lista de Strings: ["SP/São Paulo", "SP/Sorocaba"]

d['Servicos_da_Inspecao'] = ['230'] # String ou Lista de Strings

d['Qnt_de_emissoes_na_faixa'] = 1 # int

d['Emissoes_nao_autorizadas'] = 0 # int

d['Horas_de_Preparacao'] = 2 # int

d['Horas_de_Deslocamento'] = 0 # int

d['Horas_de_Execucao'] = 24 # int

d['Horas_de_Conclusao'] = 4 # int

d['Latitude'] = -23.017500 # float

d['Longitude'] = -46.884722 # float

d['Uso_de_PF'] = 'Não se aplica PF - uso apenas de formulários' # string

d['Acao_de_risco_a_vida_criada'] = 'Não' # string Sim | Não

d['Impossibilidade_acesso_online'] = '0' # string '0' | '1'

d['Reservar_Instrumentos'] = '0' #string '0' = Não | '1' = 'Sim'

d['Utilizou_algum_instrumento'] = '0'

# d['notes'] = """A intensidade de campo elétrico do 2º e 3º Harmônico não apresentou níveis fora da tolerância em nenhuma das estações medidas, somente ruído foi detectado nas monitorações desses canais. Não foi possível efetuar uma radiogoniometria com localização exata, somente uma mancha de maior probabilidade da origem do sinal, as coordenadas do transmissor principal da autorizada estava contido nessa mancha e de acordo com os items 3.4.1 a 3.4.4, os níveis de sinal diminuem conforme vamos nos afastando da localidade do transmissor autorizado.A estação mais próxima do Grande ABC, no Aeroporto de Congonhas em São Paulo, foi a que apresentou o sinal mais fraco com grande interferência do áudio na intermodulação.Em suma, não houve indícios de irregularidade no período monitorado.""" # string

# No caso de uma tabela, string formatada como csv

d['notes'] = ["""Faixa, Classe Especial, Classe A, Classe B, Classe C
                VHF-L,0,5,7,5
                VHF-H,0,12,1,0
                UHF,1,1,2,4
                FM,5,1,0,0
                RADCOM,0,0,0,0
                Outorgadas com indícios de irregularidades,1,2,3,4
              """,
              """[{"Banda":"87.800 - 108.000 MHz","Sinais":35,"Licenciados":31,"NaoLicenciados":4,"NaoLicenciaveis":0},
                  {"Banda":"470.000 - 698.000 MHz","Sinais":19,"Licenciados":19,"NaoLicenciados":0,"NaoLicenciaveis":0}]
              """]

d['uploads'] = {'path': r'D:\OneDrive - ANATEL\Monitoramento\Processos\Diogo\Mapa3\106_1.csv', 
                'filename': "Lista_de_Exceção_PMEC_Etapa_2_Sorocaba", 
                'description': "Lista de Emissões Desmascaradas e Não Licenciadas",
                'content_type': '.csv'
               }

A API do Redmine possui formatos específicos de como esses campos devem ser submetidos, validar os formatos acima e fazer essa formatação exigida pela API do Redmine é o papel da função `validar_dicionario`.

In [ ]:
#hide
show_doc(validar_dicionario)

<h4 id="validar_dicionario" class="doc_header"><code>validar_dicionario</code><a href="https://github.com/ronaldokun/fiscaliza/tree/master/fiscaliza/redmine.py#L111" class="source_link" style="float:right">[source]</a></h4>

> <code>validar_dicionario</code>(**`data_dict`**:"Dicionário de Dados ou Caminho para o arquivo .json", **`inspecao`**:"Número da Inspeção a ser relatada", **`login`**:"Login Anatel do Usuário"=*`None`*, **`senha`**:"Senha Utilizada nos Sistemas Interativos da Anatel"=*`None`*, **`fiscaliza`**:"Objeto Redmine logado, opcional ao login e senha"=*`None`*, **`teste`**:"Caso verdadeiro o Fiscaliza de Teste ( Homologação ) é utilizado"=*`True`*, **`save_path`**:"Caminho para salvar o dicionário formatado"=*`None`*)

Valida as informações de data_dict e as formata como exigido pela API do Redmine.
Opcionalmente salva o dicionário serializado como .json caso seja passado um `save_path` válido
Returns: dicionário com os dados formatados

In [ ]:
dados = validar_dicionario(d, issue, login, senha, teste=True)

In [ ]:
for k,v in dados.items():
    if k == "Html":
        v = str(v)
        v = v[:100] + '\n...\n' + v[-100:]
    console.print(f'[blue]{k} [red]-> [green]{v}')

Classe_da_Inspecao -> {'id': 89, 'value': '{"valor":"Técnica","texto":"Técnica"}'}

Tipo_de_Inspecao -> {'id': 2, 'value': '{"valor":"Uso do Espectro - Monitoração","texto":"Uso
do Espectro - Monitoração"}'}

description -> Atendimento à Ação ACAO_GR01_2021_0792 #74488, instruída pela Denúncia 
202108182925413 (7283673), 
verificação da Potência e Intensidade de Campo Elétrico da Frequência 102.9MHz e seus 
harmônicos, 
além de Radiogoniometria para checagem se o transmissor está instalado no local autorizado: 
Morro do Mombuca S/N Itatiba/SP..

Fiscal_Responsavel -> {'id': 25, 'value': 887}

Fiscais -> {'id': 26, 'value': [887]}

...

Impossibilidade_acesso_online -> {'id': 450, 'value': '0'}

Reservar_Instrumentos -> {'id': 596, 'value': '0'}

Utilizou_algum_instrumento -> {'id': 598, 'value': '0'}

notes -> |_.  Faixa                                      |_.   Classe Especial |_.   Classe A
|_.   Classe B |_.   Classe C |
|<. VHF-L                                       |>.                 0 |>.          5 |>.     
7 |>.          5 |
|<. VHF-H                                       |>.                 0 |>.         12 |>.     
1 |>.          0 |
|<. UHF                                         |>.                 1 |>.          1 |>.     
2 |>.          4 |
|<. FM                                          |>.                 5 |>.          1 |>.     
0 |>.          0 |
|<. RADCOM                                      |>.                 0 |>.          0 |>.     
0 |>.          0 |
|<. Outorgadas com indícios de irregularidades  |>.                 1 |>.          2 |>.     
3 |>.          4 |
|_.  Banda                 |_.   Sinais |_.   Licenciados |_.   Não Licenciados |_.   Não 
Licenciáveis |
|<. 87.800 - 108.000 MHz   |>.       35 |>.            31 |>.                 4 |>.          
0 |
|<. 470.000 - 698.000 MHz  |>.       19 |>.            19 |>.                 0 |>.          
0 |

uploads -> [{'path': 'D:\\OneDrive - 
ANATEL\\Monitoramento\\Processos\\Diogo\\Mapa3\\106_1.csv', 'filename': 
'Lista_de_Exceção_PMEC_Etapa_2_Sorocaba', 'description': 'Lista de Emissões Desmascaradas e 
Não Licenciadas', 'content_type': '.csv'}]

### Relatar Inspeção
A função a seguir é a mais importante do módulo porque ela de fato altera os dados na plataforma Fiscaliza.

In [ ]:
#hide
show_doc(relatar_inspecao)

<h4 id="relatar_inspecao" class="doc_header"><code>relatar_inspecao</code><a href="https://github.com/ronaldokun/fiscaliza/tree/master/fiscaliza/redmine.py#L611" class="source_link" style="float:right">[source]</a></h4>

> <code>relatar_inspecao</code>(**`inspecao`**:"Número da Inspeção a ser relatada", **`login`**:"Login Anatel do Usuário", **`senha`**:"Senha Utilizada nos Sistemas Interativos da Anatel", **`dados`**:"Dicionário já validado com os Dados a serem relatados", **`teste`**:"Indica se o relato será de teste"=*`True`*, **`parar_em`**:"String indicando até onde o relato deve ser avançado"=*`'Relatada'`*, **`substituir_relatorio`**:"Substituir o relatório criado caso houver?"=*`False`*)

Relata a inspeção `inspecao` com os dados constantes no dicionário `dados`

In [ ]:
from fiscaliza.redmine import *
relatar_inspecao(issue, login, senha, dados=dados, teste=True, parar_em="Relatada", substituir_relatorio=True)

Usuário Autenticado com Sucesso 👍

Inspeção 72145 vinculada à Ação 72088

Output()

✅ Estado Atual: Relatando

🗑  Foi solicitado a substituição do Relatório, é preciso atualizar a inspeção para 
descartá-lo primeiramente. Aguarde...

...

 ⚠ Já existe um Relatório de Monitoramento criado, esse campo não será atualizado nesta 
chamada ⚠

Output()

✒ Assine o Relatório de Monitoramento e chame a função novamente ❗

Output()

{'Anexos': {'Lista_de_Exceção_PMEC_Etapa_2_Sorocaba': 'http://sistemashm.anatel.gov.br/fiscaliza/attachments/download/1312/Lista_de_Exce%C3%A7%C3%A3o_PMEC_Etapa_2_Sorocaba'},
 'id': '72145',
 'subject': 'INSP_GR01_2021_0715',
 'status': 'Relatando',
 'priority': 'Normal',
 'start_date': '2021-09-13',
 'due_date': '2021-12-31',
 'description': 'Atendimento da Denúncia AC202010}213075425 (6104512), \r\nverificação da Potência e Intensidade de Campo Elétrico da Frequência 105.1MHz e seus harmônicos, \r\nalém da checagem de Intermodulação e Espúrios nas frequências 450.3MHz e 750MHz.',
 'Data_de_inicio_efetivo': datetime.date(2021, 8, 11),
 'Classe_da_Inspecao': 'Técnica',
 'Tipo_de_Inspecao': 'Uso do Espectro - Monitoração',
 'Ano': '2021',
 'Coordenacao': '',
 'Numero_Sei_do_Processo': '{"numero"=>"53554.000013/2021-11", "link_acesso"=>"https://seihm.anatel.gov.br/sei/controlador.php?acao=procedimento_trabalhar&id_procedimento=1964149"}',
 'Fiscal_Responsavel': 'Ronaldo da Silva Alves Ba

O dicionário de dados possui os dados completos obrigatórios para o relato, a cada etapa os dados são atualizados caso distintos do que já consta no fiscaliza:
* `Rascunho` para `Aguardando Execução`
* `Aguardando Execução` para `Em Andamento`
* `Em Andamento` para `Relatando`
* `Relatando` para `Relatada`

Após o relatório ser assinado basta chamar a função com os mesmos argumentos para que a etapa final seja realizada.

In [ ]:
fiscaliza = auth_user(login, senha, True)

In [ ]:
fiscaliza.issue.update(issue, custom_fields=[{"id": 544, "value": ""}])

True

In [ ]:
#relato = relatar_inspecao(issue, login, senha, dados=dados, parar_em="Relatada")

Os mesmos passos acima podem ser efetuados no **Fiscaliza** de produção, bastando passar o argumento `teste=False` nas funções acima

In [ ]:
# inspecao = '51849'
# detalhes = detalhar_issue(inspecao, login, senha, teste=False)
# console.print(detalhes)